In [1]:
!pip -q install pystac-client planetary-computer tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.7/207.7 kB 6.0 MB/s eta 0:00:00


In [3]:
# Choose a city by name from presets OR enter a custom bounding box.
# Format: [minLon, minLat, maxLon, maxLat]

CITY = "Bengaluru"

PRESETS = {
    "Bengaluru": [77.45, 12.80, 77.75, 13.10],
    "Hyderabad": [78.30, 17.20, 78.65, 17.60],
    "Chennai":   [80.15, 12.85, 80.35, 13.20],
    "Delhi":     [76.84, 28.40, 77.40, 28.90],
}

# If CITY="Custom", fill your own bbox here:
CUSTOM_BBOX = [77.55, 12.90, 77.65, 13.00]

if CITY == "Custom":
    BBOX = CUSTOM_BBOX
else:
    BBOX = PRESETS[CITY]

BBOX


[77.45, 12.8, 77.75, 13.1]

In [4]:
import os, json, requests
from tqdm import tqdm
from pystac_client import Client
import planetary_computer

# Create output folder
os.makedirs("ms_buildings_tiles", exist_ok=True)

# Connect to Planetary Computer STAC and search
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(collections=["ms-buildings"], bbox=BBOX)
items = list(search.get_items())
print(f"Found {len(items)} tiles for {CITY} within {BBOX}")

# Download each tile's GeoJSON
downloaded = []
for item in tqdm(items):
    asset = item.assets.get("geojson")
    if asset is None:
        continue
    url = planetary_computer.sign(asset.href)  # signed URL
    fname = f"ms_buildings_tiles/{item.id}.geojson"
    if not os.path.exists(fname):
        r = requests.get(url)
        r.raise_for_status()
        with open(fname, "wb") as f:
            f.write(r.content)
    downloaded.append(fname)

len(downloaded), downloaded[:3]


/usr/local/lib/python3.12/dist-packages/pystac_client/item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


Found 7 tiles for Bengaluru within [77.45, 12.8, 77.75, 13.1]


100%|██████████| 7/7 [00:00<00:00, 57681.98it/s]


(0, [])

In [5]:
import glob

merged = {"type": "FeatureCollection", "features": []}

for path in glob.glob("ms_buildings_tiles/*.geojson"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        # Each file is a FeatureCollection
        feats = data.get("features", [])
        merged["features"].extend(feats)

out_name = f"{CITY.lower()}_buildings.geojson"
with open(out_name, "w", encoding="utf-8") as f:
    json.dump(merged, f)

print(f"✅ Saved {len(merged['features'])} buildings to {out_name}")


✅ Saved 0 buildings to bengaluru_buildings.geojson


In [6]:
import json, glob

merged = {"type": "FeatureCollection", "features": []}

for path in glob.glob("ms_buildings_tiles/*.geojson"):
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
        feats = data.get("features", [])
        merged["features"].extend(feats)

out_name = "bengaluru_buildings.geojson"
with open(out_name, "w", encoding="utf-8") as f:
    json.dump(merged, f)

print(f"✅  Merged {len(merged['features'])} buildings into {out_name}")


✅  Merged 0 buildings into bengaluru_buildings.geojson


In [7]:
from google.colab import files
files.download("bengaluru_buildings.geojson")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
!pip -q install geopandas rasterio shapely requests tqdm matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 80.4 MB/s eta 0:00:00


In [9]:
import geopandas as gpd

CITY = "Bengaluru"
buildings = gpd.read_file("bengaluru_buildings.geojson")
print(f"✅ Loaded {len(buildings)} building footprints for {CITY}.")
buildings.head()


✅ Loaded 0 building footprints for Bengaluru.


,geometry


In [11]:
import os

if len(resp["results"]) > 0:
    image_download_url = tile["uuid"]
    # In real code, we’d fetch the GeoTIFF URL. For some OAM entries, "uuid" links to metadata page.
    # Let’s safely handle that.
    print("🔗 You can open this in browser to manually download the GeoTIFF:")
    print(f"https://openaerialmap.org/#/meta/{image_download_url}")


🔗 You can open this in browser to manually download the GeoTIFF:
https://openaerialmap.org/#/meta/https://oin-hotosm-temp.s3.amazonaws.com/62f3291df526a800059dc270/0/62f3291df526a800059dc271.tif
